In [ ]:
## information resources

# CDS database fasta generated by NCBI Nucleotide
cds="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/PaePhages_CDSs.fasta"
# ICTV VMR metadata export for Pseudomonas phages
meta="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/ICTV_Metadata_raw.txt"
# Virus-host DB
hosts="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/virushostdb.tsv"

## result files
nhmmer="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/Hmmer1/GNAT/ICTV_nhmmer_GNAT.out.table"
phmmer="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/Hmmer2/phmmer.out"
supfam="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/SUPERFAMILY_interpro/Results_IDs"
hhblits="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/HHblits/counts.n3"
phmmer_revisit="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/RevisitWithTP/phmmer/phmmer.TP.out"

## Loading databases & defining auxiliary gathering function

In [ ]:
from Bio import SeqIO

In [ ]:
seq_db = SeqIO.to_dict(SeqIO.parse(cds, "fasta"))

In [ ]:
with open(meta, "r") as meta_handle:
    meta_db = meta_handle.read()

In [ ]:
with open(hosts, "r") as hosts_handle:
    hosts_db = hosts_handle.read().split("\n")[:-1]

In [ ]:
## Collects the protein sequences of all CDS accessions in res_db in a new fasta file and composes a metadata report
# @INPUTS
# res_db: dictionary with the fasta name as key, and SeqRecord objects as values;
#         similarly as the output of the to_dict() function of SeqIO
# new_fasta_name: name of the new fasta file with all collected sequences
# protein_list_name: name of the metadata report text file
def gatherSequences(res_db, new_fasta_name, protein_list_name):
    with open(protein_list_name, "w") as list_handle:
        # header
        list_handle.write("# Species\tSpecies accession\tProtein ID\tAnnotation\tHost\tLength\tLocation\tStrand\n")
        records=[]
        for d in res_db.keys():
            # The description field of the CDS fasta contains many metadata that are ready to scrape
            description = '|'.join(res_db[d].description.split('|')[1:])
            # protein annotation
            protein =!echo "$description" | grep -Eo "\[protein=[^]]+?\]" | tr -d "[]" | cut -c 9-
            protein = protein[0]
            # genomic location
            location =!echo "$description" | grep -Eo "[0-9]+\.\.[0-9]+)?\]" | tr -d "])" | tr -s '.' '\t'
            location = [int(i) for i in location[0].split('\t')]
            # size in AA
            length = str(int((location[1] - location[0]) / 3))
            location = str(location)
            # which DNA strand
            strand =!echo "$description" | grep "complement"
            if len(strand) == 1:
                strand = "-"
            else:
                strand = "+"
            # Protein accession
            protein_id =!echo "$description" | grep -Eo "protein_id=[A-Z|0-9]+\.[0-9]" | cut -c 12-
            protein_id = protein_id[0]
            # Species accession
            species_accession =!echo "$description" | grep -Eo "[A-Z|0-9]+?\.[0-9]_prot" | cut -d '.' -f 1
            species_accession = species_accession[0]
            # Common phage name
            species =!echo -e "$meta_db" | grep "$species_accession" | cut -f 10
            species = species[0]
            # Official phage name
            classification_name = !echo -e "$meta_db" | grep "$species_accession" | cut -f 9
            classification_name = classification_name[0]
            # Phage host
            host = [i for i in hosts_db if species in i or classification_name in i]
            if len(host) == 0:
                species = species.replace("phage", "virus")
                host = [i for i in hosts_db if species in i or classification_name in i]
                if len(host) == 0:
                    print(species + " not in hosts database")
                else:
                    host = host[0]
                    host =!echo -e "$host" | cut -f 9
                    host = host[0]
            else:
                host = host[0]
                host =!echo -e "$host" | cut -f 9
                host = host[0]
            
            # write line in metadata report text file
            list_handle.write(species + "\t" + species_accession + "\t" + protein_id + "\t" + protein + "\t" \
                              + host + "\t" + length + "\t" + location + "\t" + strand + "\n")
            
            # create fasta record in memory
            new_id = species.split(' ')[2] + "_" + str(location).replace(', ','-').replace('[','').replace(']','') \
                        + " (" + species_accession + ")"
            record = SeqIO.SeqRecord(res_db[d].seq, id = new_id, name = new_id, description = protein_id)
            records.append(record)
    
    # Sort metadata entries by phage name
    !grep '#' $protein_list_name > header
    !grep -v '#' $protein_list_name | sort > tmp
    !rm $protein_list_name
    !cat header tmp > $protein_list_name
    !rm tmp
    !rm header

    # Write fasta file from records in memory
    with open(new_fasta_name, "w") as handle:
        SeqIO.write(records, handle, "fasta")

### SUPERFAMILY

In [ ]:
# Extract CDS keys from result file
with open(supfam, "r") as supfam_handle:
    supfam_keys = supfam_handle.read().split("\n")[:-1]

In [ ]:
# Select the SeqRecord instances from the full database using the CDS keys
supfam_seq_db = {i: seq_db[i] for i in supfam_keys}
supfam_seq_db

In [ ]:
# Generate multi-fasta and metadata report text file
gatherSequences(supfam_seq_db, "supfam.fasta", "protein_list_supfam")
!cat protein_list_supfam

### phmmer

In [ ]:
with open(phmmer, "r") as phmmer_handle:
    phmmer_keys_raw = phmmer_handle.read()
    phmmer_keys =!echo -e "$phmmer_keys_raw" | grep -v "#" | cut -f 1
    phmmer_keys = phmmer_keys[:-1]

In [ ]:
phmmer_seq_db = {i : seq_db[i] for i in phmmer_keys}
phmmer_seq_db

In [ ]:
gatherSequences(phmmer_seq_db, "phmmer.fasta", "protein_list_phmmer")
!cat protein_list_phmmer

 ### HHblits

In [ ]:
with open(hhblits, "r") as hhblits_handle:
    hhblits_keys_raw = hhblits_handle.read()
    hhblits_keys =!echo -e "$hhblits_keys_raw" | grep -v '#' | cut -f 1 | cut -d '.' -f 1-3
    hhblits_keys = hhblits_keys[:-1]

In [ ]:
hhblits_seq_db = {i: seq_db[i] for i in hhblits_keys}
hhblits_seq_db

In [ ]:
gatherSequences(hhblits_seq_db, "hhblits.fasta", "protein_list_hhblits")
!cat protein_list_hhblits

### nhmmer

In [ ]:
# Species accessions for all Pseudomonas phages
all_species_acc = '/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/ICTV_Metadata_PaePhageIDs.txt'

In [ ]:
### nhmmer hits are assigned to the CDS with which they have the most overlap in genomic location.

## getting genomic locations of all CDSs
# Read species accessions
with open(all_species_acc, "r") as spec_acc_handle:
    species = spec_acc_handle.read().split("\n")[:-1]
with open(cds, "r") as cds_db_handle:
    # Read full CDS fasta
    cds_db_full = cds_db_handle.read().split("\n")[:-1]
    # Extract headers
    cds_db = [i for i in cds_db_full if '>' in i]
    # Forget full CDS fasta to clear up memory
    del cds_db_full
    
    # Create dictionary of lists of genomic locations by phage species accession
    intervals = {}
    for s in species:
        # Get all CDS header strings for this species
        cds_db_spec = [i for i in cds_db if s in i]
        if len(cds_db_spec) == 0:
            continue
        # Convert list of CDS header strings into multi-line string
        cds_db_str = '\n'.join(cds_db_spec)
        # Extract genomic locations and create list of genomic intervals for this species
        location_raw =!echo "$cds_db_str" | grep -Eo "[0-9]+\.\.[0-9]+)?\]" | tr -d "])" | tr -s '.' '\t'
        location = [[int(i) for i in j.split("\t")] for j in location_raw]
        # add dictionary entry for this species
        intervals[s] = location

## getting genomic locations of nhmmer hits
with open(nhmmer, "r") as nhmmer_handle:
    nhmmer_raw = nhmmer_handle.read()
    # Extract nhmmer hit species accessions
    nhmmer_spec =!echo -e "$nhmmer_raw" | grep -v '#' | tr -s ' ' '\t' | cut -f 1
    nhmmer_spec = nhmmer_spec[:-1]
    # Get start and end point of each nhmmer hit
    nhmmer_int_start =!echo -e "$nhmmer_raw" | grep -v '#' | tr -s ' ' '\t' | cut -f 7
    nhmmer_int_start = [int(i) for i in nhmmer_int_start[:-1]]
    nhmmer_int_end =!echo -e "$nhmmer_raw" | grep -v '#' | tr -s ' ' '\t' | cut -f 8
    nhmmer_int_end = [int(i) for i in nhmmer_int_end[:-1]]
    # Convert into list of genomic intervals, turn around intervals of the complementary strand if required
    nhmmer_int_list = [sorted([nhmmer_int_start[i], nhmmer_int_end[i]]) for i in range(len(nhmmer_int_start))]
    
## match genomic intervals by greatest overlap
nhmmer_keys=[]
for i in range(len(nhmmer_spec)):
    # find genomic intervals of the species of this nhmmer hit
    try:
        cds_locs = intervals[nhmmer_spec[i]]
    except KeyError:
        print("No CDS found for " + nhmmer_spec[i])
        continue
    # determine overlap for all CDSs of that species
    cds_overlap = [min(nhmmer_int_list[i][1], cds_locs[j][1]) - max(nhmmer_int_list[i][0], cds_locs[j][0]) \
                   for j in range(len(cds_locs))]
    # convert genomic location of CDS with largest overlap to NCBI Nucleotide-style genomic location string
    cds_loc = '..'.join([str(i) for i in cds_locs[cds_overlap.index(max(cds_overlap))]])
    # link to its CDS key via its header string
    cds_key_header = [i for i in cds_db if cds_loc in i][0]
    cds_key =!echo "$cds_key_header" | cut -d ' ' -f 1 | tr -d '>'
    # add to the list of assigned CDS keys
    nhmmer_keys.append(cds_key[0])

In [ ]:
nhmmer_seq_db = {i: seq_db[i] for i in nhmmer_keys}
nhmmer_seq_db

In [ ]:
gatherSequences(nhmmer_seq_db, "nhmmer.fasta", "protein_list_nhmmer")
!cat protein_list_nhmmer

### All together

In [ ]:
all_keys = list(set(supfam_keys + phmmer_keys + hhblits_keys + nhmmer_keys))

In [ ]:
all_seq_db = {i: seq_db[i] for i in all_keys}
all_seq_db

In [ ]:
gatherSequences(all_seq_db, "all.fasta", "protein_list_all")
!cat protein_list_all

In [ ]:
!grep -v '#' protein_list_all | wc -l

Substracting the 31 known AceT, this make 81 newly found hits!

### Find new hits

First remove the initial set of AceT already known from Gene by Protein IDs.

In [ ]:
!cat ./Hmmer1/nhmmer.in.prot_IDs

In [ ]:
!cut -f 3 protein_list_all | tail -n +2 | sort > tmp1
!sort ./Hmmer1/nhmmer.in.prot_IDs > tmp2
!comm -23 tmp1 tmp2 > all.new
!rm tmp1
!rm tmp2
!cat all.new

In [ ]:
!cut -f 3 protein_list_nhmmer | tail -n +2 | sort > tmp1
!sort ./Hmmer1/nhmmer.in.prot_IDs > tmp2
!comm -23 tmp1 tmp2 > nhmmer.new
!rm tmp1
!rm tmp2
!cat nhmmer.new

In [ ]:
!cut -f 3 protein_list_phmmer | tail -n +2 | sort > tmp1
!sort ./Hmmer1/nhmmer.in.prot_IDs > tmp2
!comm -23 tmp1 tmp2 > phmmer.new
!rm tmp1
!rm tmp2
!cat phmmer.new

In [ ]:
!cut -f 3 protein_list_supfam | tail -n +2 | sort > tmp1
!sort ./Hmmer1/nhmmer.in.prot_IDs > tmp2
!comm -23 tmp1 tmp2 > supfam.new
!rm tmp1
!rm tmp2
!cat supfam.new

In [ ]:
!cut -f 3 protein_list_hhblits | tail -n +2 | sort > tmp1
!sort ./Hmmer1/nhmmer.in.prot_IDs > tmp2
!comm -23 tmp1 tmp2 > hhblits.new
!rm tmp1
!rm tmp2
!cat hhblits.new

### phmmer revisit using TPs only

In [ ]:
with open(phmmer_revisit, "r") as phmmer_revisit_handle:
    phmmer_revisit_keys_raw = phmmer_revisit_handle.read()
    phmmer_revisit_keys =!echo -e "$phmmer_revisit_keys_raw" | grep -v "#" | cut -f 1
    phmmer_revisit_keys = phmmer_revisit_keys[:-1]

In [ ]:
phmmer_revisit_seq_db = {i: seq_db[i] for i in phmmer_revisit_keys}
phmmer_revisit_seq_db

In [ ]:
gatherSequences(phmmer_revisit_seq_db, "phmmer.revisit.fasta", "protein_list_phmmer_revisit")
!cat protein_list_phmmer_revisit

In [ ]:
!comm -23 protein_list_all protein_list_phmmer_revisit

In [ ]:
!comm -13 protein_list_all protein_list_phmmer_revisit

In [ ]:
!wc -l protein_list_phmmer_revisit
!wc -l protein_list_all

### Preparing a TP-only fasta

In [ ]:
!cut -f 3 protein_list_all | tail -n +2 | sort > tmp1
!sort false.positives | comm -23 tmp1 - > all.TP
!rm tmp1
!cat all.TP

In [ ]:
!wc -l all.TP

In [ ]:
TP_accession =!cat all.TP
cds_accession = [i.split("_")[2] for i in list(seq_db.keys())]
TP_indices = [i for i in range(len(cds_accession)) if cds_accession[i] in TP_accession]
all_TP_keys = [list(seq_db.keys())[i] for i in TP_indices]
all_TP_seq_db = {i: seq_db[i] for i in all_TP_keys}
all_TP_seq_db

In [ ]:
gatherSequences(all_TP_seq_db, "all.TP.fasta", "protein_list_all_TP")
!cat protein_list_all_TP

### Preparing a TP-only fasta after the phmmer revisit

In [ ]:
!cut -f 3 protein_list_all | tail -n +2 | sort > tmp1
!comm -13 protein_list_all protein_list_phmmer_revisit | cut -f 3 > tmp2
!sort false.positives | comm -23 tmp1 - > tmp3
!cat tmp3 tmp2 > all.TP.revisit
!rm tmp1
!rm tmp2
!rm tmp3
!cat all.TP.revisit

In [ ]:
!wc -l all.TP.revisit

In [ ]:
!wc -l false.positives

In [ ]:
TP_revisit_accession =!cat all.TP.revisit
TP_revisit_indices = [i for i in range(len(cds_accession)) if cds_accession[i] in TP_revisit_accession]
all_TP_revisit_keys = [list(seq_db.keys())[i] for i in TP_revisit_indices]
all_TP_revisit_seq_db = {i: seq_db[i] for i in all_TP_revisit_keys}
all_TP_revisit_seq_db

In [ ]:
gatherSequences(all_TP_revisit_seq_db, "all.TP.revisit.fasta", "protein_list_all_TP_revisit")
!cat protein_list_all_TP_revisit